<a href="https://colab.research.google.com/github/kyle-cech/Homework4/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
drive.mount('/content/drive') # create connection to Google Drive
filedir = '/content/drive/MyDrive/'
fname = filedir + 'Wcr_GPPdaily.csv' #get name of file
df = pd.read_csv(fname) #convert csv to pandas dataframe
df = df.rename(columns = {
    'TA_F' : 'TA',
    'SW_IN_F' : 'SW',
    'VPD_F' : 'VPD',
    'GPP_NT_VUT_REF' : 'GPP'
}) #give the colums new names
df = df.drop('TIMESTAMP', axis = 1) # ger rid of TIMESTAMP column
df = df.replace(-9999.0, pd.NA)
df = df.dropna() # remove any rows with invalid data
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.56824


In [2]:
df['SW*VPD'] = df['SW'] * df['VPD']
df['SW*TA'] = df['SW'] * df['TA']
df['VPD*TA'] = df['VPD'] * df['TA'] # calculate new interaction term columns in df

In [6]:
sw = df['SW']
vpd = df['VPD']
ta = df['TA']
sw_vpd = df['SW*VPD']
sw_ta = df['SW*TA']
vpd_ta = df['VPD*TA']
gpp = df['GPP'] # assign all necessary columns in df to variables

def gpp_fun(param, sw, vpd, ta, sw_vpd, sw_ta, vpd_ta, gpp): #mse minimization function
  b0, b1, b2, b3, b4, b5, b6 = param # set coefficients equal to param
  pred = b0 + b1 * sw + b2 * vpd + b3 * ta + b4 * sw_vpd + b5 * sw_ta + b6 * vpd_ta # linear regression function
  mse = np.mean((pred - gpp)**2) # calculation for mse
  return mse # return calculated mse

guess = [10] * 7
min = minimize(gpp_fun, guess, args = (sw, vpd, ta, sw_vpd, sw_ta, vpd_ta, gpp))  # minimize linear regression function's error
b0, b1, b2, b3, b4, b5, b6 = min.x # find coefficients

rss = 0
tss = 0
total = 0
mean_gpp = df['GPP'].mean() # get mean of gpp
for row in df.itertuples(): # calculate r2 for minimization method coefficients
  rss += (row.GPP - (b0 + b1 * row.SW + b2 * row.VPD + b3 * row.TA + b4 * row._5 + b5 * row._6 + b6 * row._7)) ** 2 # find rss, squared sum of differences between real and predicted values
  tss += (row.GPP - mean_gpp) ** 2 # find tss, squared sum of differences between actual and average values
r2 = 1 - rss / tss # find r2 from rss and tss
print('intercept:', b0, 'coef:', b1, b2, b3, b4, b5, b6, 'r2:', r2)

int: 0.2555439257943212 coef: 0.017147946635719113 -0.7875601270371622 -0.013443174211800546 -0.0028758743690305015 0.0015631611987452381 0.04385533010856332 r2: 0.6827760490206342


In [4]:
x = df[['SW', 'VPD', 'TA', 'SW*VPD', 'SW*TA', 'VPD*TA']] #set x as predictors
y = gpp # set y as gpp
mod = LinearRegression()
mod.fit(x, y) #fit linear regression model
print('intercept:', mod.intercept_, 'coef:', mod.coef_, 'r2:', mod.score(x, y))

int: 0.2556079499068429 coef: [ 0.01714709 -0.78754081 -0.0134535  -0.00287589  0.00156326  0.04385347] r2: 0.6827760492480225


In [5]:
#Comparing Models
#Both models have extremely similar coefficients, intercepts, and r2 values that only slightly differ when going to 4 decimal points.
#It shows that both methods are very accurate and will get the same results.